In [1]:
import pandas as pd
import numpy as np
import os
np.random.seed(42)

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.datasets import fetch_mldata
mnist = fetch_mldata('MNIST original', data_home=os.path.join(os.environ["HOME"],"datasets"))

In [3]:
enc = OneHotEncoder()
onehot_target = enc.fit_transform( mnist.target.reshape(-1,1)).toarray()

X_train, X_test, y_train, y_test = train_test_split(mnist.data,
                                                    onehot_target,
                                                    test_size=0.25, random_state=3)
#     mnist.target

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD, RMSprop
import keras

batch_size = 128 #Number of images used in each optimization step
nb_classes = 10 #One class per digit
nb_epoch = 5 #Number of times the whole data is used to learn

dropout = 0.2
model = Sequential()
# Dense(64) is a fully-connected layer with 64 hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 20-dimensional vectors.
model.add(Dense(512, input_dim=784, init='uniform', activation='relu'))
model.add(Dropout(dropout))
model.add(Dense(512, init='uniform', activation='relu'))
model.add(Dropout(dropout))
model.add(Dense(10, init='uniform', activation='softmax'))

# sgd = SGD(lr=.1, decay=1e-6, momentum=0.9, nesterov=True)
rms = RMSprop()
model.compile(loss='categorical_crossentropy',
              optimizer=rms,
              metrics=['accuracy'])

In [7]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.loss = []
        self.accuracy = []

    def on_batch_end(self, batch, logs={}):
        self.loss.append(logs.get('loss'))
        self.accuracy.append(logs.get('acc'))

In [8]:
history = LossHistory()
model.fit(X_train, y_train,
          nb_epoch=nb_epoch,
          batch_size=batch_size,
          validation_data=(X_test, y_test),
          callbacks=[history])

score = model.evaluate(X_test, y_test, batch_size=batch_size)

Train on 52500 samples, validate on 17500 samples
Epoch 1/5
52500/52500 [==============================] - 10s - loss: 10.0148 - acc: 0.3770 - val_loss: 9.7036 - val_acc: 0.3975
Epoch 2/5
52500/52500 [==============================] - 10s - loss: 9.9818 - acc: 0.3800 - val_loss: 9.9729 - val_acc: 0.3808
Epoch 3/5
52500/52500 [==============================] - 14s - loss: 10.0558 - acc: 0.3756 - val_loss: 9.9400 - val_acc: 0.3831
Epoch 4/5
52500/52500 [==============================] - 16s - loss: 10.0004 - acc: 0.3792 - val_loss: 9.9279 - val_acc: 0.3838
Epoch 5/5
17408/17500 [============================>.] - ETA: 0s